In [1]:
import pandas as pd
import random
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [2]:
#Hasta 15 de Abril (para entrenar)

postulantes_edu_df = pd.read_csv('fiuba_1_postulantes_educacion_h15.csv')
postulantes_gye_df = pd.read_csv('fiuba_2_postulantes_genero_y_edad_h15.csv')
vistas_df = pd.read_csv('fiuba_3_vistas_h15.csv')
postulaciones_df = pd.read_csv('fiuba_4_postulaciones_h15.csv')
avisos_online_df = pd.read_csv('fiuba_5_avisos_online_h15.csv')
avisos_detalle_df = pd.read_csv('fiuba_6_avisos_detalle_h15.csv')

In [3]:
#Originales TP1 (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas.csv')
postulaciones_df2 = pd.read_csv('fiuba_4_postulaciones.csv')
avisos_online_df2 = pd.read_csv('fiuba_5_avisos_online.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle.csv')

In [4]:
#Realizo el Merge del tipo outer de los datos
#Originales TP1 y Datos hasta 15 de abril

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
postulaciones_df = postulaciones_df.merge(postulaciones_df2,how='outer')
avisos_online_df = avisos_online_df.merge(avisos_online_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')

In [5]:
#Desde 15 de abril (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion_d15.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad_d15.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas_d15.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle_d15.csv')

In [6]:
#Avisos detalle que faltaban
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [7]:
#Mergeamos todo

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_missing,how='outer')

# Libero Espacio de la Memoria

In [8]:
del postulantes_edu_df2
del postulantes_gye_df2
del vistas_df2
del avisos_detalle_df2

# Filtrado de Postulantes con informacion completa

In [9]:
postulantes_edu_df.head()

,idpostulante,nombre,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [10]:
postulantes_gye_df.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [11]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

In [12]:
postulantes_edu_df.estado.value_counts()

Graduado      451142
En Curso      182338
Abandonado     60161
Name: estado, dtype: int64

In [13]:
postulantes_edu_df.nombre.value_counts()

Universitario        250850
Secundario           244689
Terciario/Técnico    113421
Otro                  53363
Posgrado              20624
Master                10074
Doctorado               620
Name: nombre, dtype: int64

In [14]:
print(postulantes_edu_df.shape)
print(postulantes_gye_df.shape)

(693641, 3)
(505382, 3)


In [15]:
print(postulantes_edu_df.idpostulante.unique().shape)
print(postulantes_gye_df.idpostulante.unique().shape)

(447909,)
(504407,)


In [16]:
postulantes_edu_df['estado'].value_counts()

Graduado      451142
En Curso      182338
Abandonado     60161
Name: estado, dtype: int64

In [17]:
def setCategory(c):
    if c == 'Graduado':
        return 2
    elif c == 'En Curso':
        return 1
    return 0 

In [18]:
postulantes_edu_df['valor_estado'] = postulantes_edu_df['estado'].apply(setCategory)

In [19]:
postulantes_edu_df.head()

,idpostulante,nombre,estado,valor_estado
0,ZjlZ,Master,En Curso,1
1,NdJl,Posgrado,En Curso,1
2,5kNq,Otro,En Curso,1
3,8rYD,Master,En Curso,1
4,1Wvj,Universitario,En Curso,1


In [20]:
postulantes_edu_df['nombre'].value_counts()

Universitario        250850
Secundario           244689
Terciario/Técnico    113421
Otro                  53363
Posgrado              20624
Master                10074
Doctorado               620
Name: nombre, dtype: int64

In [21]:
def setCategory(c):
    
    if c == 'Doctorado':
        return 12
    elif c == 'Master':
        return 10
    elif c == 'Posgrado':
        return 8
    elif c == 'Universitario':
        return 6
    elif c == 'Terciario/Técnico':
        return 4
    elif c == 'Secundario':
        return 2
    return 0 

In [22]:
postulantes_edu_df['valor_nombre'] = postulantes_edu_df['nombre'].apply(setCategory)

In [23]:
postulantes_edu_df.head()

,idpostulante,nombre,estado,valor_estado,valor_nombre
0,ZjlZ,Master,En Curso,1,10
1,NdJl,Posgrado,En Curso,1,8
2,5kNq,Otro,En Curso,1,0
3,8rYD,Master,En Curso,1,10
4,1Wvj,Universitario,En Curso,1,6


In [24]:
postulantes_edu_df['valoracion_edu'] = postulantes_edu_df['valor_estado'] + postulantes_edu_df['valor_nombre']
postulantes_edu_df.drop(columns=['valor_estado','valor_nombre','nombre','estado'],axis=1,inplace=True)
postulantes_edu_df.sort_values(by='valoracion_edu',ascending=False,inplace=True)
postulantes_edu_df.drop_duplicates(inplace=True)
postulantes_edu_df.head()

,idpostulante,valoracion_edu
318724,N9AN45,14
128733,vV9Ewjj,14
337840,aMbWjD,14
398665,OqmK81N,14
14753,JBrVvBp,14


In [25]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

In [26]:
def setCategory(c):
    if c == '0.0':
        return 'NO_DECLARA'
    return c

In [27]:
postulantes_gye_df['sexo'] = postulantes_gye_df['sexo'].apply(setCategory)

In [28]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25943
Name: sexo, dtype: int64

In [29]:
#Para la informacion faltante agregamos una Categoria no declara
postulantes_filtrados_df = postulantes_gye_df.merge(postulantes_edu_df,on='idpostulante',how='outer')

print(postulantes_filtrados_df.shape)

postulantes_filtrados_df['idpostulante'].unique().shape

(738053, 4)


(504407,)

In [30]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,NaN
1,Nzz,NaN,NO_DECLARA,NaN
2,ZX1,NaN,NO_DECLARA,NaN
3,Nq5,NaN,NO_DECLARA,NaN
4,ebE,1952-07-07,MASC,10.0


In [31]:
#PONGAMOS EDAD PROMEDIO.


postulantes_filtrados_df['sexo'].fillna('NO_DECLARA',inplace=True)
postulantes_filtrados_df['valoracion_edu'].fillna(0, inplace=True)
postulantes_filtrados_df['fechanacimiento'].fillna('1918-01-01', inplace=True)

In [32]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,0.0
1,Nzz,1918-01-01,NO_DECLARA,0.0
2,ZX1,1918-01-01,NO_DECLARA,0.0
3,Nq5,1918-01-01,NO_DECLARA,0.0
4,ebE,1952-07-07,MASC,10.0


In [33]:
postulantes_filtrados_df.shape

(738053, 4)

In [34]:
postulantes_filtrados_df.sort_values(by='sexo',inplace=True)

In [35]:
#Sacamos los repetidos de postulantes_filtrados_df

postulantes_filtrados_df = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes_filtrados_df.shape

(504407, 4)

In [36]:
postulantes_filtrados_df.sexo.value_counts()

FEM           251226
MASC          227832
NO_DECLARA     25349
Name: sexo, dtype: int64

In [37]:
postulantes_filtrados_df.idpostulante.unique().shape

(504407,)

In [38]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,valoracion_edu
369026,5mPYWjp,1989-05-01,FEM,4.0
588433,xkdxkDz,1993-08-28,FEM,5.0
303865,EzE23E4,1999-04-23,FEM,4.0
303863,6r5OwJO,1997-06-07,FEM,4.0
588434,lDL3D1X,1997-07-17,FEM,0.0


In [39]:
postulantes_filtrados_df.fechanacimiento.describe()

count         504407
unique         17247
top       1918-01-01
freq           26080
Name: fechanacimiento, dtype: object

In [40]:
#Obtengo la edad
postulantes_edad_trucha = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == True]
postulantes_edad_trucha['fechanacimiento'] = '1918-01-01'
postulantes_edad_trucha.head()


C:\Users\Andres\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,idpostulante,fechanacimiento,sexo,valoracion_edu
299748,8M2R6pz,1918-01-01,FEM,0.0
311804,0zkAXBE,1918-01-01,FEM,7.0
341475,VNrP8EE,1918-01-01,FEM,0.0
347580,xkdB9JK,1918-01-01,FEM,8.0
324043,VNxMRxj,1918-01-01,FEM,2.0


In [41]:
postulantes_filtrados_df = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == False]
postulantes_filtrados_df = postulantes_filtrados_df.append(postulantes_edad_trucha)
postulantes_filtrados_df.head()


,idpostulante,fechanacimiento,sexo,valoracion_edu
369026,5mPYWjp,1989-05-01,FEM,4.0
588433,xkdxkDz,1993-08-28,FEM,5.0
303865,EzE23E4,1999-04-23,FEM,4.0
303863,6r5OwJO,1997-06-07,FEM,4.0
588434,lDL3D1X,1997-07-17,FEM,0.0


In [42]:
postulantes_filtrados_df.shape

(504407, 4)

In [43]:
postulantes_filtrados_df.loc[:,'fechanacimiento'] = pd.to_datetime(postulantes_filtrados_df['fechanacimiento']) 
postulantes_filtrados_df.loc[:,'edad'] = (2018 - postulantes_filtrados_df.fechanacimiento.dt.year)
postulantes_filtrados_df.drop(columns='fechanacimiento',axis=1,inplace=True)
postulantes_filtrados_df.head()
#Generar edad por bloques, como queremos hacer con las empresas mas populares.

,idpostulante,sexo,valoracion_edu,edad
369026,5mPYWjp,FEM,4.0,29
588433,xkdxkDz,FEM,5.0,25
303865,EzE23E4,FEM,4.0,19
303863,6r5OwJO,FEM,4.0,21
588434,lDL3D1X,FEM,0.0,21


In [44]:
postulantes_filtrados_df.head()

,idpostulante,sexo,valoracion_edu,edad
369026,5mPYWjp,FEM,4.0,29
588433,xkdxkDz,FEM,5.0,25
303865,EzE23E4,FEM,4.0,19
303863,6r5OwJO,FEM,4.0,21
588434,lDL3D1X,FEM,0.0,21


# Avisos Detalle

In [45]:
avisos_detalle_df.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [46]:
print(avisos_detalle_df.shape)
print(avisos_detalle_df['idaviso'].unique().shape)

(25901, 11)
(25288,)


In [47]:
#Sacamos los repetidos de avisos_detalle
avisos_detalle_filtrados_df = avisos_detalle_df.drop_duplicates('idaviso')
avisos_detalle_df.shape

(25901, 11)

In [48]:
#No se usa mas, a partir de ahora usamos avisos detalle filtrados.
del avisos_detalle_df

In [49]:
#Me quedo con los avisos del archivo de postulaciones. Esto seria, solo los avisos que tuvieron postulaciones.


#¿Aca me podria quedar con los archivos que NO tuvieron postulaciones para armar un set de NO postulaciones?

#column = postulaciones_df['idaviso'].to_frame()
#avisos_detalle_filtrados_df = column.drop_duplicates('idaviso').merge(avisos_detalle_df,on='idaviso',how='inner')
#avisos_no_postulaciones_df = pd.merge(avisos_detalle_df,avisos_detalle_filtrados_df,on='idaviso',how='right') 
#IMPLEMENTAR
avisos_detalle_filtrados_df.shape

(25288, 11)

Hago la conversion de variables categoricas a variables numericas

In [50]:
#Hago un drop de las columnas que no voy a utilizar.
#Mas adelanto debo modificar esto porque la idea es usar denominacion empresa, descripcion y titulo para nuevos features.

avisos_detalle_filtrados_df= avisos_detalle_filtrados_df.drop(columns=['idpais','mapacalle','ciudad','descripcion','titulo'],axis=1)


In [51]:
avisos_detalle_filtrados_df.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


# Postulaciones

In [52]:
postulantes_filtrados_df['idpostulante'].unique().shape

(504407,)

In [53]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [54]:
#Comparamos los "usuarios" con los  postulantes.

print(postulantes_filtrados_df.idpostulante.unique().shape)
print(postulaciones_df.idpostulante.unique().shape)

(504407,)
(348883,)


In [55]:
#Como resultado tenemos los postulantes que coinciden en postulaciones y postulantes
#Observamos que hay postulaciones de usuarios que no estan en el dataframe de postulantes.
postulantes_filtrados_efectivos_df = postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')
postulantes_filtrados_efectivos_df.idpostulante.unique().shape

(348883,)

In [56]:
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.drop_duplicates('idpostulante')
postulantes_filtrados_efectivos_df.shape

(348883, 6)

In [57]:
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop(['idaviso','fechapostulacion'],axis='columns')
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.reset_index()
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop('index',axis='columns')
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,valoracion_edu,edad
0,NjlD,FEM,6.0,37
1,ZaO5,FEM,6.0,40
2,NdJl,MASC,8.0,49
3,5kNq,MASC,1.0,45
4,Ez8J,MASC,6.0,50


In [58]:
# postulantes_filtrados_efectivos_df tiene los postulantes que efectivamente se postularon
# y que ademas tienen informacion sobre edad y estudios, etc.
# Los que estan en postulaciones pero NO en gye y edu, fueron descartados en este paso.
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,valoracion_edu,edad
0,NjlD,FEM,6.0,37
1,ZaO5,FEM,6.0,40
2,NdJl,MASC,8.0,49
3,5kNq,MASC,1.0,45
4,Ez8J,MASC,6.0,50


In [59]:
postulantes_filtrados_efectivos_df.shape

(348883, 4)

# Me fijo si las vistas implican una no postulacion

In [60]:
#vistas_df.head()

In [61]:
#vistas_df.columns=['idaviso','timestamp','idpostulante']

In [62]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [63]:
#Efectivamente una "vista" no implica una no postulacion.
#postulaciones_df.merge(vistas_df,on=['idaviso','idpostulante'],how='inner').head()

# Armado del Set de Entrenamiento

In [64]:
# Primero armo un dataframe con la informacion de postulaciones que ya tengo -> postulantes_df
# Observar que aca puede haber postulantes sobre los cuales no tengo informacion sobre genero y edad.
# Tambien puede haber postulantes que luego no aparezcan en el test final.


# ¿Deberia filtrar los postulantes sobre los cuales no tengo info sobre su genero y edad?
avisos = avisos_detalle_filtrados_df
avisos = avisos['idaviso'].to_frame()

postulantes = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes = postulantes['idpostulante'].to_frame()


postulantes_df =  postulaciones_df.merge(avisos,how='inner',on='idaviso')
postulantes_df = postulantes_df.merge(postulantes,how='inner',on='idpostulante')

postulantes_df =postulantes_df.groupby('idpostulante').agg({'idaviso':'value_counts'})
postulantes_df.columns=['postulacion']
postulantes_df['postulacion'] = 1 #Para los casos en que se repiten.

avisos = 0
postulantes = 0

In [65]:
postulantes_df.head()

postulacion
idpostulante idaviso                
0z5Dmrd      1112384041            1
             1112420060            1
0z5JW1r      1112257171            1
             1112281879            1
             1112292169            1

In [66]:
postulantes_df.reset_index(inplace=True)

In [67]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [68]:
postulantes_df.idaviso.unique().shape

(18695,)

In [69]:
#avisos_detalle_df['idaviso'].unique().shape

In [70]:
#Armado de NO postulaciones

#Verificar que las listas utilizadas sean las correctas. Estamos usando solo avisos con postulacion.

#¿Podriamos usar los avisos que no tienen postulacion?


lista = []
user_list = postulantes_filtrados_df['idpostulante'].unique() #Utilizo a todos los usuarios. 
aviso_list = avisos_detalle_filtrados_df['idaviso'].unique() #Utilizo todos los avisos.
index = 1
for n in range(10000):
    for i in range(500):
        user_index = random.randint(0,504371)
        aviso_index = random.randint(0,25287)
        lista.append((index,user_list[user_index],aviso_list[aviso_index],0))
        index+=1
#Utilizar las vistas que no se postularon como no postulaciones.
        

In [71]:
no_postulantes_df = pd.DataFrame(lista)

In [72]:
#postulantes_filtrados_df=0
#avisos_detalle_df= 0
#avisos_detalle_filtrados_df=0
#avisos_online_df=0
#postulaciones_df=0
del lista

In [73]:
no_postulantes_df.memory_usage(deep=True).sum()

438942912

In [74]:
no_postulantes_df.head()

,0,1,2,3
0,1,8xG5rO,1112427544,0
1,2,MVPYZGv,1112224474,0
2,3,1QPK81B,1111973180,0
3,4,6rPl18O,1112271156,0
4,5,qevOmOb,1112291690,0


In [75]:
no_postulantes_df.drop(0,axis='columns',inplace=True)

In [76]:
no_postulantes_df.head()

,1,2,3
0,8xG5rO,1112427544,0
1,MVPYZGv,1112224474,0
2,1QPK81B,1111973180,0
3,6rPl18O,1112271156,0
4,qevOmOb,1112291690,0


In [77]:
no_postulantes_df.columns= ['idpostulante','idaviso','postulacion']

In [78]:
no_postulantes_df.shape

(5000000, 3)

In [79]:
no_postulantes_df.head()

,idpostulante,idaviso,postulacion
0,8xG5rO,1112427544,0
1,MVPYZGv,1112224474,0
2,1QPK81B,1111973180,0
3,6rPl18O,1112271156,0
4,qevOmOb,1112291690,0


In [80]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [88]:
#Junto el dataframe donde tengo los postulantes con el aviso al que se postularon
#Con el dataframe generado al azar.
entrenamiento_df = no_postulantes_df.merge(postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [89]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion_x,postulacion_y
0,8xG5rO,1112427544,0.0,NaN
1,MVPYZGv,1112224474,0.0,NaN
2,1QPK81B,1111973180,0.0,NaN
3,6rPl18O,1112271156,0.0,NaN
4,qevOmOb,1112291690,0.0,NaN


In [90]:
entrenamiento_df['postulacion_x'].fillna(0,inplace=True)
entrenamiento_df['postulacion_y'].fillna(0,inplace=True)

In [91]:
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion_x'] + entrenamiento_df['postulacion_y']

In [92]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion_x,postulacion_y,postulacion
0,8xG5rO,1112427544,0.0,0.0,0.0
1,MVPYZGv,1112224474,0.0,0.0,0.0
2,1QPK81B,1111973180,0.0,0.0,0.0
3,6rPl18O,1112271156,0.0,0.0,0.0
4,qevOmOb,1112291690,0.0,0.0,0.0


In [93]:
 entrenamiento_df.drop(['postulacion_x','postulacion_y'],axis='columns',inplace=True)

In [94]:
entrenamiento_df['postulacion'].value_counts()

1.0    6187929
0.0    4997492
Name: postulacion, dtype: int64

In [95]:
entrenamiento_df['postulacion'] = pd.Categorical(entrenamiento_df['postulacion'])
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion'].cat.codes
entrenamiento_df['postulacion'].value_counts()

1    6187929
0    4997492
Name: postulacion, dtype: int64

In [96]:
# Hasta aca Ya tenemos nuestro formato (id,postulante,aviso,postulacion)
entrenamiento_df.head()

,idpostulante,idaviso,postulacion
0,8xG5rO,1112427544,0
1,MVPYZGv,1112224474,0
2,1QPK81B,1111973180,0
3,6rPl18O,1112271156,0
4,qevOmOb,1112291690,0


In [97]:
entrenamiento_df.tail()

,idpostulante,idaviso,postulacion
11185416,zvxzrWm,1112438823,1
11185417,zvxzrWm,1112442213,1
11185418,zvxzrWm,1112445486,1
11185419,zvxzrWm,1112448822,1
11185420,zvxzrWm,1112455515,1


In [98]:
entrenamiento_df.idaviso.unique().shape

(25288,)

# Armo la Matriz de Features

In [99]:
del postulantes_edu_df
del postulantes_gye_df
del postulantes_edad_trucha
del avisos_detalle_missing
del postulaciones_df2

In [100]:
#Para levantar los csv guardados previamente.


#entrenamiento_df=pd.read_csv('entrenamiento_df_1')
#entrenamiento_df.drop('Unnamed: 0',axis=1,inplace=True)
#avisos_detalle_filtrados_df=pd.read_csv('avisos_detalle_filtrados_df')
#postulantes_filtrados_df=postulantes_filtrados_nuevo


In [101]:
#Aca me quedo con solo 100000 datos del set de entrenamiento, sino me tarda demasiado




entrenamiento_df = entrenamiento_df.sample(1000000)

In [102]:
#Realizo un merge de postulaciones_df ampliando con la informacion de cada aviso.

#Me quedo con las postulaciones sobre las cuales tengo informacion sobre los avisos.

merge_detalle_postulaciones_df = postulaciones_df.merge(avisos_detalle_filtrados_df, on = 'idaviso',how='inner')

In [103]:
merge_detalle_postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112248724,NjlD,2018-01-19 07:39:16,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
1,1112248724,MV6eM8N,2018-01-30 23:44:30,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
2,1112248724,5LzQkN,2018-01-15 07:23:49,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
3,1112248724,8M2rOBz,2018-01-30 22:20:45,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack
4,1112248724,Dr6V3Da,2018-01-23 15:11:35,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack


In [115]:
#En primer lugar agrupo todos los avisos a los que se postulo un usuario

grupo_postulantes = merge_detalle_postulaciones_df.groupby('idpostulante')


In [126]:
#En segundo lugar hago que 'idaviso' sea el indice para poder acceder mas facil.

grupo_avisos = avisos_detalle_filtrados_df.groupby('idaviso')


In [127]:
grupo_avisos.head(3)

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD
5,1112211493,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Mantenimiento,BAYTON
6,1112211531,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Administración,Manpower
7,1112211553,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Educación/ Docentes,High Education
8,1112211555,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Control de Gestión,Grupo Gestión
9,1112211559,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Atención al Cliente,RANDSTAD


In [128]:
#Armo lista de los usuarios que se postularon a algun aviso del que yo tenga datos en avisos_detalle_filtrados_df

usuarios_con_postulaciones = grupo_postulantes.groups.keys()

In [130]:
grupo_postulantes.get_group('0zBB1W9')['nombre_area'].value_counts()

Almacén / Depósito / Expedición    1
Gastronomia                        1
Name: nombre_area, dtype: int64

In [131]:
#Cuento las postulaciones por area de cada postulante sobre el cual tengo informacion
def armarFeaturePostulacionesUsuarioAEseArea(idpostulante,idaviso):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    area = grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    areas = grupo_postulantes.get_group(idpostulante)['nombre_area'].value_counts()
    if areas.index.contains(area):
        return areas[area]
    return 0

In [ ]:
#Tarda mucho, por eso me quedé solo con 100000 datos del entrenamiento_df
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseArea(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
((pd.DataFrame(feature))[0]==entrenamiento_df['postulacion']).value_counts()

In [ ]:
entrenamiento_df['postulaciones_a_ese_area'] = pd.DataFrame(feature)

In [ ]:
def armarFeaturePostulacionesUsuarioAEsaEmpresa(idpostulante,idaviso):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    empresa = grupo_avisos.get_group(idaviso)['denominacion_empresa'].iloc[0]
    empresas = grupo_postulantes.get_group(idpostulante)['denominacion_empresa'].value_counts()
    if empresas.index.contains(empresa):
        return empresas[empresa]
    return 0

In [ ]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
entrenamiento_df['postulaciones_a_esa_empresa'] = pd.DataFrame(feature)

In [ ]:
matrix_df = entrenamiento_df

In [ ]:
matrix_df.head()

In [ ]:
matrix_df.idaviso.shape

In [ ]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

In [ ]:
#Ver que está ocupando memoria.
%who

In [ ]:
#Me quedo con el indice como "id" de la tupla postulante-aviso.
matrix_df.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

In [ ]:
matrix_df.head()

In [ ]:
del entrenamiento_df

# Test Final DF

In [ ]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [ ]:
test_final_df.head()

In [ ]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseArea(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
test_final_df['postulaciones_a_ese_area']=pd.DataFrame(feature)

In [ ]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
test_final_df['postulaciones_a_esa_empresa']=pd.DataFrame(feature)

In [253]:
test_final_df.head()

,id,idaviso,idpostulante,postulaciones_a_ese_area,postulaciones_a_esa_empresa
0,0,739260,6M9ZQR,0,0
1,1,739260,6v1xdL,5,0
2,2,739260,ezRKm9,0,0
3,3,758580,1Q35ej,0,0
4,4,758580,EAN4J6,0,0


In [ ]:
#Chequeamos que esten todos los datos.
test_final_df.shape

In [ ]:
test_final_df = test_final_df.drop(['id','idaviso','idpostulante'],axis=1)

In [258]:
test_final_df.head()

,postulaciones_a_ese_area,postulaciones_a_esa_empresa
0,0,0
1,5,0
2,0,0
3,0,0
4,0,0


In [ ]:
matrix_df['postulacion'].value_counts()

In [ ]:
X = matrix_df

In [ ]:
y = X['postulacion']
X = X.drop('postulacion',axis=1)

# Perceptron

In [ ]:
from sklearn.linear_model import Perceptron
ptron = Perceptron()

In [ ]:
ptron.fit(X,y)

In [ ]:
pred = ptron.predict(test_final_df)

In [ ]:
#Si se quiere hacer un submit solo con perceptron, pasar directo al Armado de Submit

# Bagging con Perceptron

In [284]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=ptron,n_estimators=50)

In [285]:
bc.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linea

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linea

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linea

BaggingClassifier(base_estimator=Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [286]:
pred = bc.predict(test_final_df)

# Naive_Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
matrix_df.head()

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
X.columns

In [ ]:
#from sklearn.preprocessing import StandardScaler

#features = ['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral','nombre_area','sexo','nombre','estado','edad']
#X = X.loc[:, features].values
#X = StandardScaler().fit_transform(X)
#X

In [ ]:
#X = pd.DataFrame(X)
#X.columns = ['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral','nombre_area','sexo','nombre','estado','edad']

#X.head()

In [ ]:
#Fijarse de normalizar para knn

In [ ]:
entrenamiento_df = ""
postulantes_df = ""
no_postulantes_df = ""

In [ ]:
y.value_counts()

In [ ]:
nb.fit(X, y)

In [ ]:
nb.score(X,y)

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
pred = nb.predict(test_final_df)

In [ ]:
pred

# ARMADO DE SUBMIT

In [287]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [288]:
test_final_df['sepostulo'] = pd.DataFrame(pred)

In [289]:
submit = test_final_df.drop(['idaviso','idpostulante'],axis=1)

In [290]:
submit.set_index('id',inplace=True)

In [291]:
submit.head()

,sepostulo
id,
0,0
1,0
2,0
3,0
4,0


In [292]:
submit.sepostulo.value_counts()

0    60425
1    39575
Name: sepostulo, dtype: int64

In [294]:
submit.head()

,sepostulo
id,
0,0
1,0
2,0
3,0
4,0


In [293]:
submit.to_csv('Submit_file.csv')